In [7]:
import pandas as pd

def getReceiptMbs():
    df = pd.read_csv("data/1000i.csv",names = ['recpt_id','quantity','item'])
    
    #mbs: market baskets; maps the receipt number to a list of all the items purchased
    mbs = {}
    for row in df.values:
        item_id = row[2]
        r_id = row[0]

        if(r_id not in mbs):
            mbs[r_id] = []

        mbs[r_id].append(item_id)
    
    return mbs

In [ ]:
def apriori(mbs,itemsets,minSupp):
    #Fx all items sets of length x where the support of an item set is greater
    #than the minimum support
    #all the f arrays
    fs= []
    f1 = []
    for iset in itemsets:
        if(type(iset) is not list):
            iset = [iset]
               
        supp = getSupport(iset,mbs)
        if(supp >= minSupp):
             f1.append(iset)     
    fs.append(f1)
    k = 1
    
    while(len(fs[k-1])>0):
        #The candidates for itemsets
        pcands= candGen(fs[k-1],k-1)
        #Counts the number of market baskets that contain the potential candidate
        counts_k = []
        #Initializing all counts to 0
        for c in pcands:
            counts_k.append(0)
        
        cand_idx = 0
        for mb in mbs:
            for c in pcands:
                if(isSubset(c,mb)):
                    counts_k[cand_idx]+=1
                cand_idx+=1    
            cand_idx = 0
        
        fk = []
        
        while(cand_idx < len(pcands)):
            if(counts_k[cand_idx]/len(mbs) >= minSupp):
                fk.append(pcands[cand_idx])
            cand_idx += 1
        
        fs.append(fk)   
        k += 1
                    
    
    return fs